# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Read data from toronto_df_2.csv

### The file contains the output from the second section of the assignment

In [2]:
tor_csv = "toronto_df_2.csv" # Output file from the second section
df_toronto = pd.read_csv(tor_csv)
df_toronto

Borough                                       Neighborhood  \
0         Scarborough                                     Rouge, Malvern   
1         Scarborough             Highland Creek, Rouge Hill, Port Union   
2         Scarborough                  Guildwood, Morningside, West Hill   
3         Scarborough                                             Woburn   
4         Scarborough                                          Cedarbrae   
5         Scarborough                                Scarborough Village   
6         Scarborough        East Birchmount Park, Ionview, Kennedy Park   
7         Scarborough                    Clairlea, Golden Mile, Oakridge   
8         Scarborough    Cliffcrest, Cliffside, Scarborough Village West   
9         Scarborough                        Birch Cliff, Cliffside West   
10        Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...   
11        Scarborough                                  Maryvale, Wexford   
12        Scarborough                                          Agincourt   
13        Scarborough            Clarks Corners, Sullivan, Tam O'Shanter   
14        Scarborough  Agincourt North, L'Amoreaux East, Milliken, St...   
15        Scarborough                                    L'Amoreaux West   
16        Scarborough                                        Upper Rouge   
17         North York                                  Hillcrest Village   
18         North York                       Fairview, Henry Farm, Oriole   
19         North York                                    Bayview Village   
20         North York                           Silver Hills, York Mills   
21         North York                            Newtonbrook, Willowdale   
22         North York                                   Willowdale South   
23         North York                                    York Mills West   
24         North York                                    Willowdale West   
25         North York                                          Parkwoods   
26         North York                                    Don Mills North   
27         North York                   Flemingdon Park, Don Mills South   
28         North York    Bathurst Manor, Downsview North, Wilson Heights   
29         North York                    Northwood Park, York University   
30         North York                        CFB Toronto, Downsview East   
31         North York                                     Downsview West   
32         North York                                  Downsview Central   
33         North York                                Downsview Northwest   
34         North York                                   Victoria Village   
35          East York                    Woodbine Gardens, Parkview Hill   
36          East York                                   Woodbine Heights   
37       East Toronto                                        The Beaches   
38          East York                                            Leaside   
39          East York                                   Thorncliffe Park   
40          East York                                       East Toronto   
41       East Toronto                       The Danforth West, Riverdale   
42       East Toronto                     The Beaches West, India Bazaar   
43       East Toronto                                    Studio District   
44    Central Toronto                                      Lawrence Park   
45    Central Toronto                                   Davisville North   
46    Central Toronto                                 North Toronto West   
47    Central Toronto                                         Davisville   
48    Central Toronto                        Moore Park, Summerhill East   
49    Central Toronto  Deer Park, Forest Hill SE, Rathnelly, South Hi...   
50   Downtown Toronto                                           Rosedale   
51   Downtown Toronto                        Cabbagetown, St. 

In [3]:
print(df_toronto.size)

515


### Remove all boroughs that do NOT contain 'Toronto'

In [4]:
df_toronto = df_toronto[df_toronto.Borough.str.contains("Toronto")].reset_index(drop=True)
df_toronto

Borough                                       Neighborhood  \
0       East Toronto                                        The Beaches   
1       East Toronto                       The Danforth West, Riverdale   
2       East Toronto                     The Beaches West, India Bazaar   
3       East Toronto                                    Studio District   
4    Central Toronto                                      Lawrence Park   
5    Central Toronto                                   Davisville North   
6    Central Toronto                                 North Toronto West   
7    Central Toronto                                         Davisville   
8    Central Toronto                        Moore Park, Summerhill East   
9    Central Toronto  Deer Park, Forest Hill SE, Rathnelly, South Hi...   
10  Downtown Toronto                                           Rosedale   
11  Downtown Toronto                        Cabbagetown, St. James Town   
12  Downtown Toronto                               Church and Wellesley   
13  Downtown Toronto                          Harbourfront, Regent Park   
14  Downtown Toronto                           Ryerson, Garden District   
15  Downtown Toronto                                     St. James Town   
16  Downtown Toronto                                        Berczy Park   
17  Downtown Toronto                                 Central Bay Street   
18  Downtown Toronto                           Adelaide, King, Richmond   
19  Downtown Toronto  Harbourfront East, Toronto Islands, Union Station   
20  Downtown Toronto           Design Exchange, Toronto Dominion Centre   
21  Downtown Toronto                     Commerce Court, Victoria Hotel   
22   Central Toronto                                           Roselawn   
23   Central Toronto                Forest Hill North, Forest Hill West   
24   Central Toronto                The Annex, North Midtown, Yorkville   
25  Downtown Toronto                     Harbord, University of Toronto   
26  Downtown Toronto          Chinatown, Grange Park, Kensington Market   
27  Downtown Toronto  CN Tower, Bathurst Quay, Island airport, Harbo...   
28  Downtown Toronto                    Stn A PO Boxes 25 The Esplanade   
29  Downtown Toronto             First Canadian Place, Underground city   
30  Downtown Toronto                                           Christie   
31      West Toronto                       Dovercourt Village, Dufferin   
32      West Toronto                           Little Portugal, Trinity   
33      West Toronto       Brockton, Exhibition Place, Parkdale Village   
34      West Toronto                      High Park, The Junction South   
35      West Toronto                             Parkdale, Roncesvalles   
36      West Toronto                                 Runnymede, Swansea   
37      East Toronto  Business Reply Mail Processing Centre 969 Eastern   

   PostCode   Latitude  Longitude  
0       M4E  43.676357 -79.293031  
1       M4K  43.679557 -79.352188  
2       M4L  43.668999 -79.315572  
3       M4M  43.659526 -79.340923  
4       M4N  43.728020 -79.388790  
5       M4P  43.712751 -79.390197  
6       M4R  43.715383 -79.405678  
7       M4S  43.704324 -79.388790  
8       M4T  43.689574 -79.383160  
9       M4V  43.686412 -79.400049  
10      M4W  43.679563 -79.377529  
11      M4X  43.667967 -79.367675  
12      M4Y  43.665860 -79.383160  
13      M5A  43.654260 -79.360636  
14      M5B  43.657162 -79.378937  
15      M5C  43.651494 -79.375418  
16      M5E  43.644771 -79.373306  
17      M5G  43.657952 -79.387383  
18      M5H  43.650571 -79.384568  
19      M5J  43.640816 -79.381752  
20      M5K  43.647177 -79.381576  
21      M5L  43.648198 -79.379817  
22      M5N  43.711695 -79.416936  
23      M5P  43.696948 -79.411307  
24      M5R  43.672710 -79.405678  
25      M5S  43.662696 -79.400049  
26      M5T  43.653206 -79.400049  
27      M5V  43.628947 -79.394420  
28      M5W  43.646435 -79.374846  
29     

In [5]:
print(df_toronto.size)

190


### Get the latitude and longitude of the city of Toronto

In [6]:
city = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(city, latitude, longitude))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map showing all neighborhoods within buroughs containing "Toronto" in their name

In [7]:
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, long, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#8796af',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Set my FourSquare credentials

In [8]:
CLIENT_ID = 'my id' # Foursquare ID
CLIENT_SECRET = 'my secret' # Foursquare Secret
VERSION = '20180604' # Foursquare API version

### Let's explore the first neighborhood in our dataframe.

#### Get the neighborhood's name.

In [9]:
df_toronto.loc[0, 'Neighborhood']

'The Beaches'

#### Get the neighborhood's latitude and longitude values.

In [10]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

#### First, let's create the GET request URL. Name your URL url.

In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ADTOKVZFZPYAUL0TGHFPBJEWTMQOMZJGQLH0HFYHPKEDSLLZ&client_secret=0OKRJQNQ3ONOPIXDSJOXEFMC5JTRLQCNGYGHMAIESUTNHXBF&v=20180604&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

#### Send the GET request and examine the results

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cba0e554c1f6726f15ca467'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a52011f820e3',
       'name': 'The Big Carrot Natural Food Market',
       'location': {'address': '125 Southwood Dr',
        'lat': 43.678879,
        'lng': -79.297734,
        'labeledLatLngs': [{'label': 'display',
          'lat

#### From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we are ready to clean the json and structure it into a pandas dataframe.

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name         categories        lat        lng
0  The Big Carrot Natural Food Market  Health Food Store  43.678879 -79.297734
1                 Grover Pub and Grub                Pub  43.679181 -79.297215
2                           Starbucks        Coffee Shop  43.678798 -79.298045
3                       Upper Beaches       Neighborhood  43.680563 -79.292869
4                          Dip 'n Sip        Coffee Shop  43.678897 -79.297745

#### Now we are ready to clean the json and structure it into a pandas dataframe.

In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


### Explore Neighborhoods in Toronto

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called toronoto_venues.¶

In [17]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                 latitudes=df_toronto['Latitude'],
                                 longitudes=df_toronto['Longitude']
                                )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

#### Let's check the size of the resulting dataframe

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(1721, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  The Beaches              43.676357              -79.293031   
1  The Beaches              43.676357              -79.293031   
2  The Beaches              43.676357              -79.293031   
3  The Beaches              43.676357              -79.293031   
4  The Beaches              43.676357              -79.293031   

                                Venue  Venue Latitude  Venue Longitude  \
0  The Big Carrot Natural Food Market       43.678879       -79.297734   
1                 Grover Pub and Grub       43.679181       -79.297215   
2                           Starbucks       43.678798       -79.298045   
3                       Upper Beaches       43.680563       -79.292869   
4                          Dip 'n Sip       43.678897       -79.297745   

      Venue Category  
0  Health Food Store  
1                Pub  
2        Coffee Shop  
3       Neighborhood  
4        Coffee Shop

#### Let's check how many venues were returned for each neighborhood

In [19]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                              100   
Berczy Park                                                            57   
Brockton, Exhibition Place, Parkdale Village                           22   
Business Reply Mail Processing Centre 969 Eastern                      19   
CN Tower, Bathurst Quay, Island airport, Harbou...                     15   
Cabbagetown, St. James Town                                            50   
Central Bay Street                                                     86   
Chinatown, Grange Park, Kensington Market                             100   
Christie                                                               16   
Church and Wellesley                                                   89   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             35   
Davisville North                                                        9   
Deer Park, Forest Hill SE, Rathnelly, South Hil...                     14   
Design Exchange, Toronto Dominion Centre                              100   
Dovercourt Village, Dufferin                                           20   
First Canadian Place, Underground city                                100   
Forest Hill North, Forest Hill West                                     4   
Harbord, University of Toronto                                         35   
Harbourfront East, Toronto Islands, Union Station                     100   
Harbourfront, Regent Park                                              48   
High Park, The Junction South                                          24   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               64   
Moore Park, Summerhill East                                             2   
North Toronto West                                                     22   
Parkdale, Roncesvalles                                                 16   
Rosedale                                                                4   
Roselawn                                                                1   
Runnymede, Swansea                                                     38   
Ryerson, Garden District                                              100   
St. James Town                                                        100   
Stn A PO Boxes 25 The Esplanade                                        96   
Studio District                                                        36   
The Annex, North Midtown, Yorkville                                    24   
The Beaches                                                             5   
The Beaches West, India Bazaar                                         23   
The Danforth West, Riverdale                                           44   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide, King, Richmond                                               100   
Berczy Park                                                             57   
Brockton, Exhibition Place, Parkdale Village                            22   
Business Reply Mail Processing Centre 969 Eastern                       19   
CN Tower, Bathurst Quay, Island airport, Harbou...                      15   
Cabbagetown, St. James Town                                             50   
Central Bay Street                                                      86   
Chinatown, Grange Park, Kensington Market                              100   
Christie                                                                16   
Church and Wellesley                                                    89   
Commerce Court, Victoria Hotel     

#### Let's find out how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


### Analyze Each Neighborhood

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Adult Boutique  Afghan Restaurant  Airport  \
0            0               0                  0        0   
1            0               0                  0        0   
2            0               0                  0        0   
3            0               0                  0        0   
4            0               0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  \
0                 0                    0             0         0   
1                 0                    0             0         0   
2                 0                    0             0         0   
3                 0                    0             0         0   
4                 0                    0             0         0   

   Argentinian Restaurant  Art Gallery  Art Museum  Arts & Crafts Store  \
0                       0            0           0                    0   
1                       0            0           0                    0   
2                       0            0           0                    0   
3                       0            0           0                    0   
4                       0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  \
0                 0                   0              0          0           0   
1                 0                   0              0          0           0   
2                 0                   0              0          0           0   
3                 0                   0              0          0           0   
4                 0                   0              0          0           0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Stadium  Basketball Stadium  Beach  \
0           0       0     0    0                 0                   0      0   
1           0       0     0    0                 0                   0      0   
2           0       0     0    0                 0                   0      0   
3           0       0     0    0                 0                   0      0   
4           0       0     0    0                 0                   0      0   

   Beer Bar  Beer Store  Belgian Restaurant  Bistro  Boat or Ferry  Bookstore  \
0         0           0                   0       0              0          0   
1         0           0                   0       0              0          0   
2         0           0                   0       0              0          0   
3         0           0                   0       0              0          0   
4         0           0                   0       0              0          0   

   Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  \
0         0                     0               0        0                0   
1         0                     0               0        0                0   
2         0                     0               0        0                0   
3         0                     0               0        0                0   
4         0                     0               0        0                0   

   Building  Burger Joint  Burrito Place  Bus Line  Butcher  Café  \
0         0             0              0         0        0     0   
1         0             0              0         0        0     0   
2         0             0              0         0        0     0   
3         0             0              0         0        0     0   
4         0             0              0         0      

#### And let's examine the new dataframe size.

In [22]:
toronto_onehot.shape

(1721, 236)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                            Adelaide, King, Richmond     0.000000   
1                                         Berczy Park     0.000000   
2        Brockton, Exhibition Place, Parkdale Village     0.045455   
3   Business Reply Mail Processing Centre 969 Eastern     0.052632   
4   CN Tower, Bathurst Quay, Island airport, Harbo...     0.000000   
5                         Cabbagetown, St. James Town     0.000000   
6                                  Central Bay Street     0.011628   
7           Chinatown, Grange Park, Kensington Market     0.000000   
8                                            Christie     0.000000   
9                                Church and Wellesley     0.022472   
10                     Commerce Court, Victoria Hotel     0.000000   
11                                         Davisville     0.000000   
12                                   Davisville North     0.000000   
13  Deer Park, Forest Hill SE, Rathnelly, South Hi...     0.000000   
14           Design Exchange, Toronto Dominion Centre     0.000000   
15                       Dovercourt Village, Dufferin     0.000000   
16             First Canadian Place, Underground city     0.000000   
17                Forest Hill North, Forest Hill West     0.000000   
18                     Harbord, University of Toronto     0.000000   
19  Harbourfront East, Toronto Islands, Union Station     0.000000   
20                          Harbourfront, Regent Park     0.020833   
21                      High Park, The Junction South     0.000000   
22                                      Lawrence Park     0.000000   
23                           Little Portugal, Trinity     0.015625   
24                        Moore Park, Summerhill East     0.000000   
25                                 North Toronto West     0.045455   
26                             Parkdale, Roncesvalles     0.000000   
27                                           Rosedale     0.000000   
28                                           Roselawn     0.000000   
29                                 Runnymede, Swansea     0.000000   
30                           Ryerson, Garden District     0.000000   
31                                     St. James Town     0.000000   
32                    Stn A PO Boxes 25 The Esplanade     0.000000   
33                                    Studio District     0.027778   
34                The Annex, North Midtown, Yorkville     0.000000   
35                                        The Beaches     0.000000   
36                     The Beaches West, India Bazaar     0.000000   
37                       The Danforth West, Riverdale     0.022727   

    Adult Boutique  Afghan Restaurant   Airport  Airport Food Court  \
0         0.000000           0.000000  0.000000            0.000000   
1         0.000000           0.000000  0.000000            0.000000   
2         0.000000           0.000000  0.000000            0.000000   
3         0.000000           0.000000  0.000000            0.000000   
4         0.000000           0.000000  0.066667            0.066667   
5         0.000000           0.000000  0.000000            0.000000   
6         0.000000           0.000000  0.000000            0.000000   
7         0.000000           0.000000  0.000000            0.000000   
8         0.000000           0.000000  0.000000            0.000000   
9         0.011236           0.011236  0.000000            0.000000   
10        0.000000           0.000000  0.000000            0.000000   
11        0.000000           0.000000  0.000000            0.000000   
12        0.000000           0.000000  0.000000            0.000000   
13        0.000000           0.000000  0.000000            0.000000   
14        0.000000           0.000000  0.000000            0.000000   
15        0.000000           0.000000  0.000000            0.000000   
16        0.000000           0.000000  0.000000            0.000000   
17        0.000000           0.0

#### Let's confirm the new size

In [24]:
toronto_grouped.shape

(38, 236)

#####  Let's print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2      Farmers Market  0.04
3              Bakery  0.04
4  Seafood Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0      Breakfast Spot  0.09
1                Café  0.09
2         Coffee Shop  0.09
3  Italian Restaurant  0.05
4       Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2         Pizza Place  0.05
3             Brewery  0.05
4          Smoke Shop  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airpo

#### Let's put that into a pandas dataframe

#### First, let's write a function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                        Berczy Park           Coffee Shop   
2       Brockton, Exhibition Place, Parkdale Village        Breakfast Spot   
3  Business Reply Mail Processing Centre 969 Eastern    Light Rail Station   
4  CN Tower, Bathurst Quay, Island airport, Harbo...       Airport Service   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Café            Steakhouse   American Restaurant   
1          Cocktail Bar                   Pub           Cheese Shop   
2                  Café           Coffee Shop           Yoga Studio   
3           Yoga Studio      Recording Studio            Smoke Shop   
4        Airport Lounge      Airport Terminal       Harbor / Marina   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0       Thai Restaurant                 Hotel      Sushi Restaurant   
1            Restaurant                Bakery        Farmers Market   
2               Stadium         Burrito Place            Restaurant   
3               Brewery                   Spa        Farmers Market   
4      Sculpture Garden              Boutique         Boat or Ferry   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                Bakery                   Bar       Asian Restaurant  
1            Steakhouse                  Café     Seafood Restaurant  
2  Caribbean Restaurant          Climbing Gym              Pet Store  
3  Fast Food Restaurant         Burrito Place             Restaurant  
4                 Plane          Airport Gate                Airport

### Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 3 clusters.

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

Borough                    Neighborhood PostCode   Latitude  \
0     East Toronto                     The Beaches      M4E  43.676357   
1     East Toronto    The Danforth West, Riverdale      M4K  43.679557   
2     East Toronto  The Beaches West, India Bazaar      M4L  43.668999   
3     East Toronto                 Studio District      M4M  43.659526   
4  Central Toronto                   Lawrence Park      M4N  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0           Coffee Shop     Health Food Store   
1 -79.352188               0      Greek Restaurant           Coffee Shop   
2 -79.315572               0                  Park        Sandwich Place   
3 -79.340923               0                  Café           Coffee Shop   
4 -79.388790               4              Bus Line                  Park   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                   Pub           Wings Joint      Department Store   
1        Ice Cream Shop    Italian Restaurant             Bookstore   
2           Pizza Place             Pet Store     Food & Drink Shop   
3                Bakery   American Restaurant    Italian Restaurant   
4           Swim School           Wings Joint     Electronics Store   

         6th Most Common Venue        7th Most Common Venue  \
0            Electronics Store  Eastern European Restaurant   
1       Furniture / Home Store                 Liquor Store   
2                 Liquor Store           Light Rail Station   
3               Sandwich Place             Stationery Store   
4  Eastern European Restaurant          Dumpling Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0   Dumpling Restaurant            Donut Shop       Doner Restaurant  
1                Bakery            Sports Bar                    Spa  
2          Burger Joint         Burrito Place   Fast Food Restaurant  
3                  Bank                   Bar            Fish Market  
4            Donut Shop      Doner Restaurant                Dog Run

#### Finally, let's visualize the resulting clusters

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

### Cluster 1

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels  \
0                                         The Beaches               0   
1                        The Danforth West, Riverdale               0   
2                      The Beaches West, India Bazaar               0   
3                                     Studio District               0   
5                                    Davisville North               0   
6                                  North Toronto West               0   
7                                          Davisville               0   
9   Deer Park, Forest Hill SE, Rathnelly, South Hi...               0   
11                        Cabbagetown, St. James Town               0   
12                               Church and Wellesley               0   
13                          Harbourfront, Regent Park               0   
14                           Ryerson, Garden District               0   
15                                     St. James Town               0   
16                                        Berczy Park               0   
17                                 Central Bay Street               0   
18                           Adelaide, King, Richmond               0   
19  Harbourfront East, Toronto Islands, Union Station               0   
20           Design Exchange, Toronto Dominion Centre               0   
21                     Commerce Court, Victoria Hotel               0   
24                The Annex, North Midtown, Yorkville               0   
25                     Harbord, University of Toronto               0   
26          Chinatown, Grange Park, Kensington Market               0   
27  CN Tower, Bathurst Quay, Island airport, Harbo...               0   
28                    Stn A PO Boxes 25 The Esplanade               0   
29             First Canadian Place, Underground city               0   
30                                           Christie               0   
31                       Dovercourt Village, Dufferin               0   
32                           Little Portugal, Trinity               0   
33       Brockton, Exhibition Place, Parkdale Village               0   
34                      High Park, The Junction South               0   
35                             Parkdale, Roncesvalles               0   
36                                 Runnymede, Swansea               0   
37  Business Reply Mail Processing Centre 969 Eastern               0   

   1st Most Common Venue          2nd Most Common Venue 3rd Most Common Venue  \
0            Coffee Shop              Health Food Store                   Pub   
1       Greek Restaurant                    Coffee Shop        Ice Cream Shop   
2                   Park                 Sandwich Place           Pizza Place   
3                   Café                    Coffee Shop                Bakery   
5                   Park                 Clothing Store        Sandwich Place   
6         Clothing Store                    Coffee Shop   Sporting Goods Shop   
7            Pizza Place                 Sandwich Place          Dessert Shop   
9            Coffee Shop                            Pub           Pizza Place   
11           Coffee Shop                     Restaurant           Pizza Place   
12           Coffee Shop            Japanese Restaurant               Gay Bar   
13           Coffee Shop                         Bakery                   Pub   
14           Coffee Shop                 Clothing Store                  Café   
15           Coffee Shop                           Café            Restaurant   
16           Coffee Shop                   Cocktail Bar                   Pub   
17           Coffee Shop             Italian Restaurant       Bubble Tea Shop   
18           Coffee Shop                           Café            Steakhouse   
19           Coffee Shop                       Aquarium                 Hotel   
20           Coffee Shop                           Café                 Hotel   
21           Coffee

### Cluster 2

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue  \
8  Moore Park, Summerhill East               1                   Gym   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
8            Playground         Grocery Store           Event Space   

  5th Most Common Venue        6th Most Common Venue 7th Most Common Venue  \
8     Electronics Store  Eastern European Restaurant   Dumpling Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
8            Donut Shop      Doner Restaurant                Dog Run

### Cluster 3

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
22     Roselawn               2                Garden           Wings Joint   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
22      Department Store  Ethiopian Restaurant     Electronics Store   

          6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
22  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

   9th Most Common Venue 10th Most Common Venue  
22      Doner Restaurant                Dog Run

### Cluster 4

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue  \
10                             Rosedale               3                  Park   
23  Forest Hill North, Forest Hill West               3                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
10            Playground                 Trail           Wings Joint   
23                 Trail         Jewelry Store      Sushi Restaurant   

   5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
10      Department Store     Electronics Store  Eastern European Restaurant   
23           Wings Joint          Dessert Shop            Electronics Store   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
10          Dumpling Restaurant            Donut Shop       Doner Restaurant  
23  Eastern European Restaurant   Dumpling Restaurant             Donut Shop

### Cluster 5

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
4  Lawrence Park               4              Bus Line                  Park   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
4           Swim School           Wings Joint     Electronics Store   

         6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
4  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

  9th Most Common Venue 10th Most Common Venue  
4      Doner Restaurant                Dog Run